# Buliding Flexible Code Base

We will build a flexible code base that increases the ease of development and code-reuse. I will take bits and pieces of FASTAI library and use this as a base for our experiments.

## Let's setup dummy dataset

In [2]:

from sklearn import datasets

noise = 0.1
X_train, y_train = datasets.make_moons(n_samples=2000, noise=noise)

In [5]:
y_train

array([0, 1, 0, ..., 1, 0, 0])

In [145]:
#create dummy dataset
import torch
from torch.utils.data import Dataset,DataLoader
from torch.nn import Sequential
import torch.nn as nn
from tqdm import tqdm
class DummyDset(Dataset):
    def __init__(self,sz,noise=0.1):
        self.x,self.y = datasets.make_moons(n_samples=sz,noise=noise)
        self.x = self.x.astype('float32')
    def __getitem__(self,ind):
        return self.x[ind],self.y[ind]
    def __len__(self):
        return len(self.x)
        
        
class Dloaders:
    def __init__(self,*dls):
        self.train,self.valid = dls

In [146]:
sampl = DummyDset(100)

In [147]:
for x,y in sampl:
    print(f'{x},{x.shape}')
    print(f'{y},{y.shape}')
    break

[0.93305933 0.2283831 ],(2,)
0,()


In [148]:
trn_dset,vld_dset,tst_dset = DummyDset(2000),DummyDset(500),DummyDset(500)

In [149]:
trn_dl = DataLoader(trn_dset,shuffle=True,batch_size=50)
vld_dl = DataLoader(vld_dset,batch_size=50)

In [150]:
dls = Dloaders(trn_dl,vld_dl)

In [151]:
for x_bl,y_bl in trn_dl:
    print(x_bl.shape)
    print(y_bl.shape)
    break

torch.Size([50, 2])
torch.Size([50])


In [159]:
dmodel = Sequential(nn.Linear(2,164),
          nn.LeakyReLU(),
          nn.Linear(164,32),
          nn.LeakyReLU(),
          nn.Linear(32,2),
          )

In [153]:
dmodel

Sequential(
  (0): Linear(in_features=2, out_features=164, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=164, out_features=32, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=32, out_features=2, bias=True)
)

In [154]:
loss_fn = nn.CrossEntropyLoss()
optimizer_fn = torch.optim.Adam

In [128]:
# define Python user-defined exceptions
class CancelFitException(Exception):
    pass

In [156]:
xb,yb = None,None
for xb,yb in trn_dl:
    break
    

In [157]:
out = dmodel(xb)

## Flexible Trainer Class

In [186]:



class Trainer:
    def __init__(self,model,dls,loss_func,lr,opt_func,cbs,device='cpu'):
        self.model = model
        self.dl = dls
        self.loss_func = loss_func
        self.lr = lr
        self.opt_func = opt_func
        self.cbs =cbs
        self.device = device
        for cb in cbs:cb.trainer = self
        
    def one_batch(self,batch):
        self('before_batch')
        xb,yb = batch
        self.opt.zero_grad() 
        #storing y_preds onto the class for a potential use by a callback
        self.y_preds = self.model(xb)
        self.loss = self.loss_func(self.y_preds,yb)
        if self.model.training:
            #take backward step only if in training mode
            self.loss.backward()
            self.opt.step()
        self('after_batch')
        
    def one_epoch(self,is_train):
        self.model.training = is_train
        self('before_epoch')
        #choose the appropriate data-loader
        dl = self.dl.train if is_train else self.dl.valid
        for self.epochnum,self.batch in enumerate(dl):
            #import pdb;pdb.set_trace()
            self.one_batch(self.batch)
        self('after_epoch')
    
    def fit(self,nepochs):
        self('before_fit')
        self.opt = self.opt_func(self.model.parameters(),self.lr)
        self.nepochs = nepochs
        try:
            for self.epoch in tqdm(range(self.nepochs)):
                self.one_epoch(True)
                self.one_epoch(False)
        except CancelFitException:
            pass
        self('after_fit')
        
    def predict(self,dl):
        print('NOT IMPLEMENTED')
        pass
    
    def infer(self,x):
        pass
    
    def __call__(self,name):
        for cb in self.cbs:getattr(cb,name)()
        
        

In [187]:
class CallBack:
    def before_epoch(self):
        pass
    def after_epoch(self):
        pass
    def before_batch(self):
        pass
    def after_batch(self):
        pass
    def before_fit(self):
        pass
    def after_fit(self): 
        pass
    
    
    

class SetupTrainerCB(CallBack):
    def before_batch(self):
        xb,yb = to_device(self.trainer.device,self.trainer.batch)
        self.trainer.batch = xb,yb

    def before_fit(self): 
        if self.trainer.device != 'cpu':
            self.trainer.model.cuda()

In [188]:
class PrinterCB(CallBack):
    def before_epoch(self):
        self.ns,self.losses,self.accs = [],[],[]
        
    def after_epoch(self):
        n = sum(self.ns)
        epoch_type = 'Training' if self.trainer.model.training else 'Validation'
        acc = sum(self.accs)/n
        loss = sum(self.losses)/n
        print(f"{epoch_type} acc: {acc},loss:{loss}")
        
    def after_batch(self):
        xb,yb = self.trainer.batch
        acc = (self.trainer.y_preds.argmax(dim=-1) == yb).float().sum()
        ns = len(xb)
        l = (self.trainer.loss.item())*ns
        self.ns.append(ns)
        self.losses.append(l)
        self.accs.append(acc)
    

In [189]:
cbs = [PrinterCB()]

In [190]:
trainer = Trainer(dmodel,dls,loss_fn,0.0002,optimizer_fn,cbs,'cpu')

In [192]:
trainer.fit(10)

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 44.04it/s]

Training acc: 0.9169999957084656,loss:0.1967030894011259
Validation acc: 0.9259999990463257,loss:0.1817723885178566
Training acc: 0.9254999756813049,loss:0.17515053320676088
Validation acc: 0.9399999976158142,loss:0.15973494127392768
Training acc: 0.9375,loss:0.1553548775613308
Validation acc: 0.9480000138282776,loss:0.14143396019935608
Training acc: 0.9465000033378601,loss:0.13779674749821424
Validation acc: 0.9559999704360962,loss:0.12448145151138305
Training acc: 0.9589999914169312,loss:0.12202388374134898
Validation acc: 0.9620000123977661,loss:0.10851196274161339
Training acc: 0.965499997138977,loss:0.10720596723258495
Validation acc: 0.9660000205039978,loss:0.09475147388875485
Training acc: 0.9704999923706055,loss:0.09367854483425617
Validation acc: 0.972000002861023,loss:0.08259923197329044
Training acc: 0.9794999957084656,loss:0.0818658934906125
Validation acc: 0.9800000190734863,loss:0.07173150032758713
Training acc: 0.9829999804496765,loss:0.0715909406542778
Validation acc: 0

In [50]:
from torch import as_tensor,Tensor,ByteTensor,LongTensor,FloatTensor,HalfTensor,DoubleTensor
from types import GeneratorType

def is_listy(x):
    "`isinstance(x, (tuple,list,slice,GeneratorType))`"
    return isinstance(x, (tuple,list,slice,GeneratorType))



def apply(func, x, *args, **kwargs):
    "Apply `func` recursively to `x`, passing on args"
    if is_listy(x): return type(x)([apply(func, o, *args, **kwargs) for o in x])
    if isinstance(x,dict):  return {k: apply(func, v, *args, **kwargs) for k,v in x.items()}
    res = func(x, *args, **kwargs)
    return res 

def to_device(b, device=None, non_blocking=False):
    "Recursively put `b` on `device`."
    #if defaults.use_cuda==False: device='cpu'
    #elif device is None: device=default_device()
    def _inner(o):
        if isinstance(o,Tensor): return o.to(device, non_blocking=non_blocking)
#         if hasattr(o, "to_device"): return o.to_device(device)
        return o
    return apply(_inner, b)